<a href="https://colab.research.google.com/github/Suryadeep01/OpenAppChat/blob/main/TASK_GARUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Object list.xlsx to Object list.xlsx


In [2]:
import gdown
video_url = 'https://drive.google.com/uc?id=1twzI9bwctiOkTIS-9ljmKzp5lF4bHwnO'
video_path = '/content/WhatsApp Video 2025-05-28 at 3.15.58 PM.mp4'
gdown.download(video_url, video_path, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1twzI9bwctiOkTIS-9ljmKzp5lF4bHwnO
To: /content/WhatsApp Video 2025-05-28 at 3.15.58 PM.mp4
100%|██████████| 96.8M/96.8M [00:02<00:00, 35.3MB/s]


'/content/WhatsApp Video 2025-05-28 at 3.15.58 PM.mp4'

In [ ]:
try:
    import ultralytics
    print("ultralytics already installed")
except ImportError:
    print("Installing ultralytics...")
    !pip install ultralytics
try:
    import cv2, pandas, pytesseract, PIL, numpy, tqdm, base64, IPython
    print("Other dependencies already installed")
except ImportError:
    print("Installing other dependencies...")
    !pip install opencv-python pandas pytesseract openpyxl tqdm
try:
    import easyocr
    print("easyocr already installed")
except ImportError:
    print("Installing easyocr...")
    !pip install easyocr
!apt-get update && apt-get install -y tesseract-ocr

import cv2
from ultralytics import YOLO
import pandas as pd
import pytesseract
import numpy as np
import os
import time
import base64
from google.colab import files
from tqdm import tqdm
from IPython.display import HTML
from collections import defaultdict
import easyocr

class CentroidTracker:
    def __init__(self, max_disappeared=50):
        self.next_object_id = 0
        self.objects = defaultdict(lambda: None)
        self.bboxes = defaultdict(lambda: None)
        self.labels = defaultdict(lambda: None)
        self.confidences = defaultdict(lambda: None)
        self.disappeared = defaultdict(int)
        self.max_disappeared = max_disappeared

    def register(self, centroid, bbox, label, confidence):
        self.objects[self.next_object_id] = centroid
        self.bboxes[self.next_object_id] = bbox
        self.labels[self.next_object_id] = label
        self.confidences[self.next_object_id] = confidence
        self.disappeared[self.next_object_id] = 0
        self.next_object_id += 1

    def deregister(self, object_id):
        del self.objects[object_id]
        del self.bboxes[object_id]
        del self.labels[object_id]
        del self.confidences[object_id]
        del self.disappeared[object_id]

    def update(self, rects, labels, confidences):
        if len(rects) == 0:
            for object_id in list(self.disappeared.keys()):
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)
            return self.objects, self.bboxes, self.labels, self.confidences

        input_centroids = np.zeros((len(rects), 2), dtype="int")
        input_bboxes = []
        for i, (x, y, w, h) in enumerate(rects):
            cX = int(x)
            cY = int(y)
            input_centroids[i] = (cX, cY)
            input_bboxes.append((x, y, w, h))

        if len(self.objects) == 0:
            for i in range(len(rects)):
                self.register(input_centroids[i], input_bboxes[i], labels[i], confidences[i])
        else:
            object_ids = list(self.objects.keys())
            object_centroids = np.array(list(self.objects.values()))

            D = np.linalg.norm(input_centroids[:, np.newaxis] - object_centroids, axis=2)
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]

            used_rows = set()
            used_cols = set()

            for row, col in zip(rows, cols):
                if row in used_rows or col in used_cols:
                    continue

                object_id = object_ids[col]
                self.objects[object_id] = input_centroids[row]
                self.bboxes[object_id] = input_bboxes[row]
                self.labels[object_id] = labels[row]
                self.confidences[object_id] = confidences[row]
                self.disappeared[object_id] = 0

                used_rows.add(row)
                used_cols.add(col)

            unused_rows = set(range(len(input_centroids))).difference(used_rows)
            unused_cols = set(range(len(object_centroids))).difference(used_cols)

            for row in unused_rows:
                self.register(input_centroids[row], input_bboxes[row], labels[row], confidences[row])

            for col in unused_cols:
                object_id = object_ids[col]
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)

        return self.objects, self.bboxes, self.labels, self.confidences

print("Please upload Object list.xlsx")
uploaded = files.upload()
excel_path = '/content/Object list (1).xlsx'

for attempt in range(5):
    if os.path.exists(excel_path):
        print(f"Excel file found at {excel_path}")
        break
    print(f"Waiting for {excel_path} to appear... (Attempt {attempt+1}/5)")
    time.sleep(2)
else:
    print(f"Error: Excel file {excel_path} not found after upload")
    exit()

try:
    df_objects = pd.read_excel(excel_path, sheet_name='Sheet1')
    object_list = df_objects['Objects'].dropna().tolist()
    print("Objects loaded from Excel:", object_list)
except FileNotFoundError:
    print(f"Error: Excel file {excel_path} not found")
    exit()
except KeyError:
    print("Error: Excel file must have a column named 'Objects' in 'Sheet1'")
    exit()

video_path = '/content/WhatsApp Video 2025-05-28 at 3.15.58 PM.mp4'
if not os.path.exists(video_path):
    print(f"Error: Video file {video_path} not found")
    exit()
print(f"Video file loaded: {video_path}")

model = YOLO('yolov8m.pt')

reader = easyocr.Reader(['en'], gpu=True)

yolo_to_object = {
    'car': 'Car',
    'bus': 'Bus',
    'truck': 'Truck',
    'motorcycle': 'Motorcycle',
    'traffic light': 'Traffic Signals'
}

tracker = CentroidTracker(max_disappeared=50)
results = []

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Could not open video file {video_path}")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = min(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), 5000)

output_path_video = '/content/temp_output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path_video, fourcc, fps, (width, height))

frame_count = 0

with tqdm(total=total_frames, desc="Processing video") as pbar:
    while cap.isOpened() and frame_count < total_frames:
        ret, frame = cap.read()
        if not ret:
            break

        annotated_frame = frame.copy()
        rects, detection_labels, confidences = [], [], []

        if frame_count % 30 == 0:
            timestamp = frame_count / fps
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            enhanced = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                            cv2.THRESH_BINARY, 11, 2)
            enhanced = cv2.GaussianBlur(enhanced, (3, 3), 0)

            detections = model(frame)[0]
            for det in detections.boxes:
                label = model.names[int(det.cls)]
                if label in yolo_to_object:
                    x, y, w, h = det.xywh[0].tolist()
                    obj_name = yolo_to_object[label]
                    if label == 'car' and w * h < 3000:
                        obj_name = 'Auto'
                    rects.append((x, y, w, h))
                    detection_labels.append(obj_name)
                    confidences.append(det.conf.item())
                    results.append({
                        'Object': obj_name,
                        'Timestamp': f"{timestamp:.2f}s",
                        'Location': f"x: {x:.0f}, y: {y:.0f}, width: {w:.0f}, height: {h:.0f}",
                        'Normalized Location': f"x: {x/width:.3f}, y: {y/height:.3f}, width: {w/width:.3f}, height: {h/height:.3f}",
                        'Object ID': -1
                    })

            # OCR with EasyOCR (fallback to Tesseract)
            try:
                ocr_results = reader.readtext(frame, detail=1, paragraph=False)
            except:
                ocr_results = []
            if not ocr_results:
                ocr_data = pytesseract.image_to_data(enhanced, config='--psm 6', output_type=pytesseract.Output.DICT)
                ocr_results = [( ([x, y, x+w, y+h], text, conf/100) ) for x, y, w, h, text, conf in zip(ocr_data['left'], ocr_data['top'], ocr_data['width'], ocr_data['height'], ocr_data['text'], ocr_data['conf']) if conf > 60 and text.strip()]

            for (bbox, text, conf) in ocr_results:
                text = text.lower().strip()
                if text:
                    x1, y1 = bbox[0]
                    x2, y2 = bbox[2]
                    x, y = (x1 + x2) / 2, (y1 + y2) / 2
                    w, h = x2 - x1, y2 - y1
                    if 'km' in text or 'kilometer' in text:
                        rects.append((x, y, w, h))
                        detection_labels.append('Km-Stone')
                        confidences.append(conf)
                        results.append({
                            'Object': 'Km-Stone',
                            'Timestamp': f"{timestamp:.2f}s",
                            'Location': f"x: {x:.0f}, y: {y:.0f}, width: {w:.0f}, height: {h:.0f}",
                            'Normalized Location': f"x: {x/width:.3f}, y: {y/height:.3f}, width: {w/width:.3f}, height: {h/height:.3f}",
                            'Object ID': -1
                        })
                    elif any(word in text for word in ['sign', 'board', 'direction']):
                        rects.append((x, y, w, h))
                        detection_labels.append('Sign-Boards')
                        confidences.append(conf)
                        results.append({
                            'Object': 'Sign-Boards',
                            'Timestamp': f"{timestamp:.2f}s",
                            'Location': f"x: {x:.0f}, y: {y:.0f}, width: {w:.0f}, height: {h:.0f}",
                            'Normalized Location': f"x: {x/width:.3f}, y: {y/height:.3f}, width: {w/width:.3f}, height: {h/height:.3f}",
                            'Object ID': -1
                        })
                    elif any(word in text for word in ['fuel', 'petrol', 'gas', 'diesel', 'bpcl', 'hp', 'indian oil']):
                        rects.append((x, y, w, h))
                        detection_labels.append('Fuel-Stations')
                        confidences.append(conf)
                        results.append({
                            'Object': 'Fuel-Stations',
                            'Timestamp': f"{timestamp:.2f}s",
                            'Location': f"x: {x:.0f}, y: {y:.0f}, width: {w:.0f}, height: {h:.0f}",
                            'Normalized Location': f"x: {x/width:.3f}, y: {y/height:.3f}, width: {w/width:.3f}, height: {h/height:.3f}",
                            'Object ID': -1
                        })
                    elif any(word in text for word in ['rest', 'service area', 'food', 'dhaba']):
                        rects.append((x, y, w, h))
                        detection_labels.append('Rest-Areas')
                        confidences.append(conf)
                        results.append({
                            'Object': 'Rest-Areas',
                            'Timestamp': f"{timestamp:.2f}s",
                            'Location': f"x: {x:.0f}, y: {y:.0f}, width: {w:.0f}, height: {h:.0f}",
                            'Normalized Location': f"x: {x/width:.3f}, y: {y/height:.3f}, width: {w/width:.3f}, height: {h/height:.3f}",
                            'Object ID': -1
                        })
                    elif 'left' in text or '←' in text:
                        rects.append((x, y, w, h))
                        detection_labels.append('Left Turn')
                        confidences.append(conf)
                        results.append({
                            'Object': 'Left Turn',
                            'Timestamp': f"{timestamp:.2f}s",
                            'Location': f"x: {x:.0f}, y: {y:.0f}, width: {w:.0f}, height: {h:.0f}",
                            'Normalized Location': f"x: {x/width:.3f}, y: {y/height:.3f}, width: {w/width:.3f}, height: {h/height:.3f}",
                            'Object ID': -1
                        })
                    elif 'right' in text or '→' in text:
                        rects.append((x, y, w, h))
                        detection_labels.append('Right Turn')
                        confidences.append(conf)
                        results.append({
                            'Object': 'Right Turn',
                            'Timestamp': f"{timestamp:.2f}s",
                            'Location': f"x: {x:.0f}, y: {y:.0f}, width: {w:.0f}, height: {h:.0f}",
                            'Normalized Location': f"x: {x/width:.3f}, y: {y/height:.3f}, width: {w/width:.3f}, height: {h/height:.3f}",
                            'Object ID': -1
                        })

            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            green_mask = cv2.inRange(hsv, np.array([35, 60, 60]), np.array([80, 255, 255]))
            green_area = cv2.countNonZero(green_mask)
            if green_area > 0.2 * frame.shape[0] * frame.shape[1]:
                x, y, w, h = width//3, height//3, width//3, height//3  # Tighter ROI
                rects.append((x, y, w, h))
                detection_labels.append('Agricultural-Areas')
                confidences.append(0.9)
                results.append({
                    'Object': 'Agricultural-Areas',
                    'Timestamp': f"{timestamp:.2f}s",
                    'Location': f"x: {x:.0f}, y: {y:.0f}, width: {w:.0f}, height: {h:.0f}",
                    'Normalized Location': f"x: {x/width:.3f}, y: {y/height:.3f}, width: {w/width:.3f}, height: {h/height:.3f}",
                    'Object ID': -1
                })

            edges = cv2.Canny(gray, 120, 220)
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            buildings = [cnt for cnt in contours if cv2.contourArea(cnt) > 2000 and len(cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)) >= 4]
            if len(buildings) >= 4:
                x, y, w, h = cv2.boundingRect(np.concatenate(buildings))
                rects.append((x + w/2, y + h/2, w, h))
                detection_labels.append('Residential-Areas')
                confidences.append(0.9)
                results.append({
                    'Object': 'Residential-Areas',
                    'Timestamp': f"{timestamp:.2f}s",
                    'Location': f"x: {x:.0f}, y: {y:.0f}, width: {w:.0f}, height: {h:.0f}",
                    'Normalized Location': f"x: {x/width:.3f}, y: {y/height:.3f}, width: {w/width:.3f}, height: {h/height:.3f}",
                    'Object ID': -1
                })

            lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=70, minLineLength=130, maxLineGap=10)
            if lines is not None:
                for line in lines:
                    x1, y1, x2, y2 = line[0]
                    if abs(x1 - x2) < 10 and abs(y1 - y2) > 130:
                        x = (x1 + x2) / 2
                        y = (y1 + y2) / 2
                        w, h = 25, abs(y2 - y1)
                        rects.append((x, y, w, h))
                        detection_labels.append('Street-Lights')
                        confidences.append(0.9)
                        results.append({
                            'Object': 'Street-Lights',
                            'Timestamp': f"{timestamp:.2f}s",
                            'Location': f"x: {x:.0f}, y: {y:.0f}, width: {w:.0f}, height: {h:.0f}",
                            'Normalized Location': f"x: {x/width:.3f}, y: {y/height:.3f}, width: {w/width:.3f}, height: {h/height:.3f}",
                            'Object ID': -1
                        })
                        break

            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            bridge_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 10000 and cv2.boundingRect(cnt)[2] > 3 * cv2.boundingRect(cnt)[3] and cv2.boundingRect(cnt)[1] < height * 0.4]
            if bridge_contours:
                x, y, w, h = cv2.boundingRect(bridge_contours[0])
                rects.append((x + w/2, y + h/2, w, h))
                detection_labels.append('Bridges')
                confidences.append(0.9)
                results.append({
                    'Object': 'Bridges',
                    'Timestamp': f"{timestamp:.2f}s",
                    'Location': f"x: {x:.0f}, y: {y:.0f}, width: {w:.0f}, height: {h:.0f}",
                    'Normalized Location': f"x: {x/width:.3f}, y: {y/height:.3f}, width: {w/width:.3f}, height: {h/height:.3f}",
                    'Object ID': -1
                })

            pole_contours = [cnt for cnt in contours if 1000 < cv2.contourArea(cnt) < 3000 and cv2.boundingRect(cnt)[2] / cv2.boundingRect(cnt)[3] < 0.3]
            if pole_contours:
                x, y, w, h = cv2.boundingRect(pole_contours[0])
                rects.append((x + w/2, y + h/2, w, h))
                detection_labels.append('Electricity-Utilities')
                confidences.append(0.9)
                results.append({
                    'Object': 'Electricity-Utilities',
                    'Timestamp': f"{timestamp:.2f}s",
                    'Location': f"x: {x:.0f}, y: {y:.0f}, width: {w:.0f}, height: {h:.0f}",
                    'Normalized Location': f"x: {x/width:.3f}, y: {y/height:.3f}, width: {w/width:.3f}, height: {h/height:.3f}",
                    'Object ID': -1
                })

            lower_road = frame[int(height*0.6):, :]
            hsv_road = cv2.cvtColor(lower_road, cv2.COLOR_BGR2HSV)
            road_mask = cv2.inRange(hsv_road, np.array([0, 0, 70]), np.array([180, 30, 130]))
            kernel = np.ones((5, 5), np.uint8)
            road_mask = cv2.morphologyEx(road_mask, cv2.MORPH_OPEN, kernel)
            contours, _ = cv2.findContours(road_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            gaps = [cnt for cnt in contours if cv2.contourArea(cnt) > 7000 and cv2.boundingRect(cnt)[2] > 120]
            if gaps:
                x, y, w, h = cv2.boundingRect(gaps[0])
                x, y = x + width//2, y + int(height*0.6) + h//2
                rects.append((x, y, w, h))
                detection_labels.append('MedianOpening')
                confidences.append(0.85)
                results.append({
                    'Object': 'MedianOpening',
                    'Timestamp': f"{timestamp:.2f}s",
                    'Location': f"x: {x:.0f}, y: {y:.0f}, width: {w:.0f}, height: {h:.0f}",
                    'Normalized Location': f"x: {x/width:.3f}, y: {y/height:.3f}, width: {w/width:.3f}, height: {h/height:.3f}",
                    'Object ID': -1
                })

            side_lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=80, minLineLength=160, maxLineGap=15)
            if side_lines is not None:
                for line in side_lines:
                    x1, y1, x2, y2 = line[0]
                    angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi
                    if abs(angle) < 30 and (x1 < width * 0.1 or x1 > width * 0.9):
                        x = (x1 + x2) / 2
                        y = (y1 + y2) / 2
                        w, h = abs(x2 - x1), 60
                        rects.append((x, y, w, h))
                        detection_labels.append('ServiceRoad')
                        confidences.append(0.9)
                        results.append({
                            'Object': 'ServiceRoad',
                            'Timestamp': f"{timestamp:.2f}s",
                            'Location': f"x: {x:.0f}, y: {y:.0f}, width: {w:.0f}, height: {h:.0f}",
                            'Normalized Location': f"x: {x/width:.3f}, y: {y/height:.3f}, width: {w/width:.3f}, height: {h/height:.3f}",
                            'Object ID': -1
                        })
                        break

        objects, bboxes, labels, confidences = tracker.update(rects, detection_labels, confidences)

        for object_id in objects.keys():
            x, y, w, h = bboxes[object_id]
            x1, y1 = int(x - w/2), int(y - h/2)
            x2, y2 = int(x + w/2), int(y + h/2)
            label = labels[object_id]
            conf = confidences[object_id]
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(annotated_frame, f"{label} ID:{object_id} {conf:.2f}", (x1, y1-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            if frame_count % 30 == 0:
                for result in results[-len(rects):]:
                    if result['Object'] == label and result['Object ID'] == -1:
                        result['Object ID'] = object_id

        out.write(annotated_frame)

        frame_count += 1
        pbar.update(1)

cap.release()
out.release()

detected_objects = set(result['Object'] for result in results)
for obj in object_list:
    if obj not in detected_objects:
        results.append({
            'Object': obj,
            'Timestamp': '-',
            'Location': 'Not observed',
            'Normalized Location': '-',
            'Object ID': -1
        })

df = pd.DataFrame(results)
df = df.sort_values(by='Object')
output_path_excel = '/content/detected_objects.xlsx'
df.to_excel(output_path_excel, index=False)
print(f"Results saved to {output_path_excel}")

try:
    with open(output_path_video, 'rb') as f:
        video_data = f.read()
    video_base64 = base64.b64encode(video_data).decode('ascii')
    video_html = f"""
    <video width="640" height="360" controls>
      <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
    </video>
    """
    display(HTML(video_html))
except Exception as e:
    print(f"Error displaying video: {e}. Downloading instead.")
    files.download(output_path_video)

files.download(output_path_excel)

ultralytics already installed
Other dependencies already installed
easyocr already installed
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,552 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,264